In [307]:
import pandas as pd

In [308]:
problem_descriptions=pd.read_csv('probelm_descrption.csv')
problem_descriptions

,id,problem_description,tags,product
0,1,traffic glitch,Network Down,NaN
1,2,error,NaN,NaN
2,4,fault,NaN,NaN
3,11,masec,NaN,NaN
4,12,macsec,MACSEC,NaN
5,20,flash error,NaN,CX
6,22,macsec issue,NaN,cx
7,23,traffic glitchs,NaN,cx
8,24,traffic glitchss,NaN,cx
9,25,traffic errror,network down,cx


In [309]:
feedback=pd.read_csv('probelm_feedback.csv')
#solution_id must a json 
feedback

,id,problem_description_id,object_type_id,solution_obj_id,user_name,upvotes,downvotes,calculated_effective_percentage
0,7,25,1,MOP-X1234,khali.saran@ascendo.ai,0,1,0.0
1,6,25,1,MOP-X1234,khali.saran@ascendo.ai,1,0,0.0
2,8,12,3,FSB-W4567,ronny.randomperson@idk.com,1,0,NaN
3,9,22,5,MOP-S234,jack@salesforce.com,0,1,NaN
4,10,1,1,FSB-W4567,khali.saran@ascendo.ai,0,1,NaN
5,11,25,1,MOP-X1234,jack@salesforce.com,1,0,NaN
6,12,25,1,MOP-X1234,ronny.randomperson@idk.com,1,0,NaN
7,13,1,2,FSB-W4567,xyz@idk.com,0,1,NaN
8,14,25,1,MOP-X9876,xyz@idk.com,1,0,NaN
9,15,12,3,FSB-R2222,khali.saran@ascendo.ai,1,0,NaN


In [310]:

def make_clusters(problem_descriptions, feedback):
    
    cluster_df=pd.DataFrame(columns=['id','cluster_id','cluster_name','problem_id'])
    i=1
    #for each problem_description
    for problem in  problem_descriptions.iterrows():
        #check if problem_description_id is present in feedback table
        problem_id= problem[1].id
        
        if problem_id in list(feedback['problem_description_id']):
            #if present, make a cluster
            cluster_df.loc[i]=(i,i,'cluster group '+str(i),problem_id)
            i=i+1      
    print(cluster_df)
    return cluster_df

In [311]:
problem_clusters= make_clusters(problem_descriptions, feedback)
problem_clusters

#id-prob_description key pair instead of problem_id,

  id cluster_id     cluster_name problem_id
1  1          1  cluster group 1          1
2  2          2  cluster group 2          2
3  3          3  cluster group 3         12
4  4          4  cluster group 4         22
5  5          5  cluster group 5         23
6  6          6  cluster group 6         24
7  7          7  cluster group 7         25
8  8          8  cluster group 8         31
9  9          9  cluster group 9         32


,id,cluster_id,cluster_name,problem_id
1,1,1,cluster group 1,1
2,2,2,cluster group 2,2
3,3,3,cluster group 3,12
4,4,4,cluster group 4,22
5,5,5,cluster group 5,23
6,6,6,cluster group 6,24
7,7,7,cluster group 7,25
8,8,8,cluster group 8,31
9,9,9,cluster group 9,32


In [312]:
def make_results(problem_clusters,feedback):
    i=1
    agg_results=pd.DataFrame(columns=['id','cluster_id','solution_id','upvotes','downvotes'])
    for each_cluster in problem_clusters.iterrows():
        problem_id=each_cluster[1].problem_id 

        solutions=feedback[feedback.problem_description_id==problem_id]
        solution_ids=list(solutions['solution_obj_id'].unique())
        
        for each_solution_id in solution_ids:
            cluster_solution=solutions[solutions['solution_obj_id']==each_solution_id]
            s_id=cluster_solution.iloc[0].solution_obj_id
            total_upvotes=cluster_solution.upvotes.sum()
            total_downvotes=cluster_solution.downvotes.sum()
            agg_results.loc[i]=(i,each_cluster[1].cluster_id,s_id,total_upvotes,total_downvotes)
            i=i+1
    return agg_results


def print_results(agg_results, problem_descriptions,feedback,problem_clusters):
    print_result=pd.DataFrame(columns=['id','cluster_id','problem_description','solution_id','upvotes','downvotes'])
    i=1
    for each_result in agg_results.iterrows():
        p_id=problem_clusters['problem_id'][problem_clusters['cluster_id']==each_result[1].cluster_id].iloc[0]
        c_id=each_result[1].cluster_id
        problem=problem_descriptions['problem_description'][problem_descriptions['id']==p_id].iloc[0]
        print_result.loc[i]=(i,c_id,problem,each_result[1].solution_id,each_result[1].upvotes,each_result[1].downvotes)
        i=i+1
    return print_result

In [313]:
agg_result=make_results(problem_clusters, feedback)
answer=print_results(agg_result,problem_descriptions,feedback,problem_clusters)
answer.to_csv('results.csv')

In [314]:
print('Total Upvotes: ', agg_result.upvotes.sum())
print('Total Downvotes: ', agg_result.downvotes.sum())

Total Upvotes:  49
Total Downvotes:  32


------------------------------------------------------------------------------------------------------------------------------

Test Cases

In [ ]:
1.  1 cluster to 1 description to 1 solution and up votes are available
2.  multiple cluster with each with 1 description and one solution with up votes
3. 1 cluster to 1 description to multiple soltuions with up votes
4. muttiple clusters with with multiple solutions but no overlap of solutions
5. Multiple clusters with 1 descriptin each but solutions overlap
6. Multiple clusters with multiple solutions and overlap of solutions
7. Handling of up vote undoing and the impact of the cluster and aggregate logic